# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos: sistema LLM + RAG**

* **Nombres y matrículas:**

  * Fernando Omar Salazar Ortiz - A01796214
  * Carlos Aaron Bocanegra Buitron - A01796345
  * Luis Enrique González González - A01795338
  * Gloria María Campos García - A01422345

* **Número de Equipo: 12**


* ##### **El formato de este cuaderno de Jupyter es libre, pero debe incuir al menos las siguientes secciones:**

  * ##### **Introducción de la problemática a resolver.**
  * ##### **Sistema RAG + LLM**
  * ##### **El chatbot, incluyendo ejemplos de prueba.**
  * ##### **Conclusiones**

* ##### **Pueden importar los paquetes o librerías que requieran.**

* ##### **Pueden incluir las celdas y líneas de código que deseen.**

In [72]:
from google.colab import drive
drive.mount('/content/drive')
google_drive_path_parent = "/content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana9_Chatbot_LLM_RAG/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
#Se hacen las instalaciones necesarias
!pip install -q openai pymupdf sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 845.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00


In [9]:
#Se hacen las importaciones necesarias
from google.colab import userdata
import openai
import fitz #PyMuPDF
import tiktoken
import numpy as np
import faiss
import pickle
import json

from sentence_transformers import SentenceTransformer

In [3]:
# Se carga la API Key de OpenAI
api_key = userdata.get("OPENAI_API_KEY_PERSONAL")

if not api_key:
  raise ValueError("API key no encontrada en los secretos")

client = openai.OpenAI(api_key=api_key)

In [51]:
def extract_text_from_pdf(
    pdf_path,
    use_blocks=True,
    min_length=40,
    exclude_headers_footers=True,
    header_height=60,
    footer_height=60,
    skip_pages=None,
    start_at_page=0,
    stop_at_page=None,
    omit_texts=None
):
    """
    Extrae texto de un PDF usando PyMuPDF, ordenando bloques visuales y controlando
    rango de páginas y omisión de textos específicos.

    Args:
        pdf_path (str): Ruta del archivo PDF.
        use_blocks (bool): Si True, extrae texto por bloques visuales; si False, por página completa.
        min_length (int): Longitud mínima para conservar un bloque.
        exclude_headers_footers (bool): Si True, excluye encabezado y pie de página.
        header_height (int): Altura desde arriba a excluir (en puntos).
        footer_height (int): Altura desde abajo a excluir (en puntos).
        skip_pages (List[int] or None): Índices de páginas (base 0) a omitir.
        start_at_page (int): Página inicial (índice base 0) para empezar a procesar.
        stop_at_page (int or None): Índice de página en la que se debe detener el procesamiento.
        omit_texts (List[str] or None): Lista de textos exactos a omitir (case-insensitive, strip).

    Returns:
        List[str]: Lista de chunks de texto extraídos del PDF.
    """
    doc = fitz.open(pdf_path)
    all_chunks = []
    skip_pages = skip_pages or []
    omit_texts = [t.strip().lower() for t in (omit_texts or [])]

    # Itera desde start_at_page hasta stop_at_page (no inclusivo)
    page_range = range(start_at_page, stop_at_page if stop_at_page is not None else len(doc))

    for i in page_range:
        if i in skip_pages:
            continue

        page = doc[i]
        page_height = page.rect.height

        if use_blocks:
            blocks = page.get_text("blocks")
            blocks = sorted(blocks, key=lambda b: (b[1], b[0]))
            for b in blocks:
                x0, y0, x1, y1, text = b[:5]
                if not text or len(text.strip()) < min_length:
                    continue
                if exclude_headers_footers and (y0 < header_height or y1 > page_height - footer_height):
                    continue
                clean_text = text.strip().lower()
                if clean_text in omit_texts:
                    continue
                all_chunks.append(text.strip())
        else:
            text = page.get_text()
            if text and len(text.strip()) >= min_length:
                clean_text = text.strip().lower()
                if clean_text not in omit_texts:
                    all_chunks.append(text.strip())

    return all_chunks


def save_chunks_to_json(chunks, file_path="chunks.json"):
    """
    Guarda una lista de bloques de texto en un archivo JSON.
    Cada línea es un bloque, editable manualmente.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)
    print(f"✅ Bloques guardados en {file_path}")


def load_chunks_from_json(file_path="chunks.json"):
    """
    Carga los bloques de texto desde un archivo JSON y devuelve una lista.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        chunks = json.load(f)
    print(f"📄 Bloques cargados: {len(chunks)}")
    return chunks


def save_embeddings_to_pkl(vectors, chunks, file_path="embeddings_data.pkl"):
    """
    Guarda los vectores (embeddings) y sus textos (chunks) en un archivo .pkl.

    Args:
        vectors (List[List[float]] o np.ndarray): Embeddings generados.
        chunks (List[str]): Lista de textos originales de cada embedding.
        file_path (str): Ruta de archivo donde se guardará todo.
    """
    data = {
        "vectors": np.array(vectors).astype("float64"),
        "chunks": chunks
    }
    with open(file_path, "wb") as f:
        pickle.dump(data, f)
    print(f"✅ Embeddings guardados en {file_path}")


def load_embeddings_from_pkl(file_path="embeddings_data.pkl"):
    """
    Carga los vectores (embeddings) y los textos (chunks) desde un archivo .pkl.

    Returns:
        Tuple[np.ndarray, List[str]]: embeddings y chunks.
    """
    with open(file_path, "rb") as f:
        data = pickle.load(f)
    print(f"📂 Cargados {len(data['chunks'])} embeddings desde {file_path}")
    return data["vectors"], data["chunks"]


def embed_texts_openai(texts):
    embeddings = []
    for chunk in texts:
        response = client.embeddings.create(
            input=chunk,
            model="text-embedding-3-small"
        )
        embedding = response.data[0].embedding
        embeddings.append(embedding)
    return np.array(embeddings).astype("float32")

def create_faiss_index(embeddings):
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

In [61]:
def retrieve_and_answer_openai(
    question,
    chunks,
    index,
    chunk_embeddings,
    top_k=5,
    use_rag=True,
    print_chunks=False
):
    if use_rag:
        q_embedding = client.embeddings.create(
            input=question,
            model="text-embedding-3-small"
        ).data[0].embedding
        q_vector = np.array(q_embedding).astype("float32").reshape(1, -1)
        D, I = index.search(q_vector, top_k)
        if print_chunks:
            print("------------------------------------")
            for rank, idx in enumerate(I[0]):
                print(f"--- Chunk #{idx} --")
                print(f"Distancia (D): {D[0][rank]} \n")
                print(chunks[idx])
            print("------------------------------------")
        context = "\n---\n".join([chunks[i] for i in I[0]])

        prompt = f"""Usa la siguiente información para responder la pregunta de forma clara y completa. Solo responde con base en la información proporcionada, no busques datos en otras fuentes
Información:
{context}

Pregunta: {question}
Respuesta:"""

    else:
        prompt = f"""Responde de forma clara y completa la siguiente pregunta sobre alimentación durante el embarazo. Si no tienes información suficiente, dilo claramente.

Pregunta: {question}
Respuesta:"""

    response = client.chat.completions.create(
        #model="gpt-3.5-turbo",
        model="o4-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Eres un chatbot para resolver dudas de la alimentación durante el embarazo "
                    "que responde en base a la información proporcioanda de contexto. "
                    "Si la información no es suficiente, entonces dices que no puedes responder esa pregunta."
                )
            },
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


In [73]:
# PROCESAMIENTO

pdf_path = google_drive_path_parent + "guia-alimentaria-mujeres-embarazadas.pdf"
chunks = extract_text_from_pdf(
    pdf_path=pdf_path,
    use_blocks=True,
    min_length=3,
    exclude_headers_footers=True,
    header_height=0,
    footer_height=50,
    skip_pages=[],
    start_at_page=6,
    stop_at_page=78,
    omit_texts=["¿CÓMO ESTAMOS EN MÉXICO?",
                "¿POR QUÉ ES IMPORTANTE ESTA RECOMENDACIÓN?",
                "En la salud:",
                "En las mujeres:",
                "En las y los bebés:",
                "En la cultura y en la sociedad:",
                "En el ambiente:",
                "En la economía, en la cultura y en la sociedad:",
                "En el o la bebé gestante",
                "En mujeres embarazadas:",
                "En mujeres en periodo de lactancia y posparto:",
                "En las y los recién nacidos:",
                "ORIENTE SOBRE CÓMO PONER EN PRÁCTICA \nLA RECOMENDACIÓN, MENCIONANDO QUE:",
                ]
)

In [74]:
len(chunks)

478

In [75]:
#save_chunks_to_json(chunks=chunks, file_path="/content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana9_Chatbot_LLM_RAG/chunks.json")
save_chunks_to_json(chunks=chunks, file_path= google_drive_path_parent + "chunks.json")

✅ Bloques guardados en /content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana9_Chatbot_LLM_RAG/chunks.json


In [76]:
chunks2 =load_chunks_from_json(file_path=google_drive_path_parent + "chunks_cambios_manuales.json")
#Los cambios manuales que hicieron fueron eliminar la información correspondiente a las tablas

📄 Bloques cargados: 293


In [34]:
#Se generan los embeddings
#embeddings = embed_texts_openai(chunks2)

In [77]:
#save_embeddings_to_pkl(embeddings, chunks2, file_path=google_drive_path_parent + "embeddings_guia_alimentaria.pkl")

✅ Embeddings guardados en /content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana9_Chatbot_LLM_RAG/embeddings_guia_alimentaria.pkl


In [81]:
embeddings, chunks2 =load_embeddings_from_pkl(file_path=google_drive_path_parent + "embeddings_guia_alimentaria.pkl")

📂 Cargados 293 embeddings desde /content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana9_Chatbot_LLM_RAG/embeddings_guia_alimentaria.pkl


In [36]:
#Se crea el indice
faiss_index = create_faiss_index(embeddings)

In [84]:
#pregunta = "¿Se puede consumir café durante el embarazo?"
#pregunta = "¿Se puede consumir alcohol durante el embarazo?"
#pregunta = "¿Cuánta agua se debe consumir durante el embarazo"
#pregunta = "¿Qué alimentos y bebidas se deben evitar durante el embarazo?"
#pregunta = "¿cuántas leguminosas se recomienda consumir durante el embarazo?"
#pregunta = "¿Qué frutas puedo comer durante el embarazo?" # No tiene buenos resultados esta pregunta
#pregunta = "¿Qué cereales debo comer durante el embarazo?"
#pregunta = "¿Puedo comer carne de pescado?"
#pregunta = "¿Puedo comer elote?"
pregunta = "¿Puedo comer frutos rojos, como fresas, zarzamoras, arandanos, etc.?"
#respuesta = retrieve_and_answer(pregunta, chunks, faiss_index, embeddings)
respuesta = retrieve_and_answer_openai(pregunta, chunks2, faiss_index, embeddings, use_rag=True, print_chunks=True)
print("\n🤖 Respuesta del chatbot:\n", respuesta)

------------------------------------
--- Chunk #66 --
Distancia (D): 0.9339300990104675 

Pruebe congelar tus frutas 
favoritas, ya sea plátano, 
fresas o uvas, entre otras.
--- Chunk #65 --
Distancia (D): 0.9717541337013245 

Incluya frutas de temporada, tanto 
en el desayuno, los refrigerios o en 
cualquier momento del día en el que 
tenga hambre o antojo de algo dulce.
--- Chunk #71 --
Distancia (D): 0.988568902015686 

Evite consumir frutas con azúcar, miel, en almíbar 
o con grasas añadidas como crema o media
crema. Si las consume, que sea ocasionalmente.
--- Chunk #64 --
Distancia (D): 1.0223766565322876 

PARA COMER MÁS FRUTAS
--- Chunk #136 --
Distancia (D): 1.0291736125946045 

Recuerde no exceder el consumo de carnes rojas y evitar las carnes procesadas.
------------------------------------

🤖 Respuesta del chatbot:
 Sí. Los frutos rojos (fresas, zarzamoras, arándanos, etc.) son una excelente opción de fruta. Para aprovecharlos al máximo:

1. Congélalos: puedes picarlos y met

# **Conclusiones:**

* #### **Incluyan sus conclusiones de la actividad chatbot LLM + RAG:**



None

# **Fin de la actividad chatbot: LLM + RAG**